# KoBERT finetuning

In [2]:
#pip install ipywidgets  # for vscode
#!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
#!pip install mxnet==1.6.0 gluonnlp==0.9.1 sentencepiece==0.1.91 pandas==1.0.5 transformers==2.11.0 pytorch_lightning==0.8.1

^C


In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

C:\Users\green\anaconda3\envs\cp3\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
#from kobert import get_tokenizer
#from kobert import get_pytorch_kobert_model

In [5]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
#import urllib
import pandas as pd

In [8]:
## CPU
#device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [18]:
#l, vocab = get_pytorch_kobert_model(cachedir=".cache")
#bertmodel, vocab = get_kobert_model(cachedir=".cache")

In [30]:
#kobert
from KoBERT.kobert.utils import get_tokenizer
from KoBERT.kobert.pytorch_kobert import get_kobert_model
#tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
#tokenizer = get_tokenizer()
#tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
#bertmodel, vocab = get_kobert_model('skt/kobert-base-v1', tokenizer.vocab_file)

ImportError: cannot import name 'get_kobert_model' from 'KoBERT.kobert.pytorch_kobert' (C:\Users\green\Desktop\workspace\colorpoem\KoBERT\kobert\pytorch_kobert.py)

In [ ]:
#!wget -O .cache/ratings_train.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
#!wget -O .cache/ratings_test.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt

--2022-05-16 13:59:07--  http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
Resolving skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)... 52.219.58.42
Connecting to skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)|52.219.58.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘.cache/ratings_train.txt’

.cache/ratings_trai 100%[===================>]  13.95M  22.7MB/s    in 0.6s    

2022-05-16 13:59:07 (22.7 MB/s) - ‘.cache/ratings_train.txt’ saved [14628807/14628807]

--2022-05-16 13:59:08--  http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt
Resolving skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)... 52.219.58.42
Connecting to skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)|52.219.58.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M

In [ ]:
#dataset_train = nlp.data.TSVDataset(".cache/ratings_train.txt", field_indices=[1,2], num_discard_samples=1)
#dataset_test = nlp.data.TSVDataset(".cache/ratings_test.txt", field_indices=[1,2], num_discard_samples=1)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
#data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
#train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
#test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
data = pd.read_csv('multiemo2.CSV', encoding='cp949')
data[:5]

,대분류,소분류,문장1,문장2
0,기쁨,신이 난,지금 난 기분이 너무 좋아.,오늘부터 연휴가 시작됐거든. 내일 회사 안 간다.
1,기쁨,신이 난,나도 조카가 생겨! 너무 기뻐.,응. 언니에게 꽃다발을 선물할 거야. 예쁜 딸이면 좋을 것 같아서.
2,불안,불안,거래처와의 다음 계약이 무산될까봐 불안해.,딱히 이유는 없어. 그냥 나 혼자 불안한 상태야.
3,슬픔,염세적인,당뇨로 고생 중인데 아내가 이혼하자더군. 아내가 날 버리면 병은 더 악화될 텐데 세...,젊은 시절에 가정에 소홀했지. 그래도 그렇지 이혼을 하자니 너무 허망해.
4,슬픔,마비된,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.,정말이지 막막해.


In [ ]:
data.loc[(data['대분류'] == "기쁨"), '대분류'] = 0 # 기쁨 = 0
data.loc[(data['대분류'] == "분노"), '대분류'] = 1 # 분노 = 1
data.loc[(data['대분류'] == "슬픔"), '대분류'] = 2 # 슬픔 = 2
data.loc[(data['대분류'] == "불안"), '대분류'] = 3 # 불안 = 3
data.loc[(data['대분류'] == "상처"), '대분류'] = 4 # 상처 = 4
data.loc[(data['대분류'] == "당황"), '대분류'] = 5 # 당황 = 5
data[:5]

,대분류,소분류,문장1,문장2
0,0,신이 난,지금 난 기분이 너무 좋아.,오늘부터 연휴가 시작됐거든. 내일 회사 안 간다.
1,0,신이 난,나도 조카가 생겨! 너무 기뻐.,응. 언니에게 꽃다발을 선물할 거야. 예쁜 딸이면 좋을 것 같아서.
2,3,불안,거래처와의 다음 계약이 무산될까봐 불안해.,딱히 이유는 없어. 그냥 나 혼자 불안한 상태야.
3,2,염세적인,당뇨로 고생 중인데 아내가 이혼하자더군. 아내가 날 버리면 병은 더 악화될 텐데 세...,젊은 시절에 가정에 소홀했지. 그래도 그렇지 이혼을 하자니 너무 허망해.
4,2,마비된,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.,정말이지 막막해.


In [ ]:
data.isnull().sum()

대분류    0
소분류    0
문장1    0
문장2    0
dtype: int64

In [ ]:
data_list = []
for ques, label in zip(data['문장1'], data['대분류'])  :
    data = []   
    data.append(ques)
    data.append(str(label))
    data_list.append(data)
data_list[:5]

[['지금 난 기분이 너무 좋아.', '0'],
 ['나도 조카가 생겨! 너무 기뻐.', '0'],
 ['거래처와의 다음 계약이 무산될까봐 불안해.', '3'],
 ['당뇨로 고생 중인데 아내가 이혼하자더군. 아내가 날 버리면 병은 더 악화될 텐데 세상에 나 혼자 남은 기분이야.', '2'],
 ['노년에 재정적으로 이루어 놓은게 없어서 걱정이야.', '2']]

In [ ]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=25)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 25 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6, # 2->6
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/936 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9016813039779663 train acc 0.15625
epoch 1 batch id 201 loss 0.9287683963775635 train acc 0.5794465174129353
epoch 1 batch id 401 loss 0.9522178769111633 train acc 0.6639650872817955
epoch 1 batch id 601 loss 0.9941439628601074 train acc 0.6865380615640599
epoch 1 batch id 801 loss 0.7256550788879395 train acc 0.695751404494382
epoch 1 train acc 0.6983112373737373


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 25 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/234 [00:00<?, ?it/s]

epoch 1 test acc 0.5948539886039885


  0%|          | 0/936 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.6462752819061279 train acc 0.78125
epoch 2 batch id 201 loss 0.7630637288093567 train acc 0.7259017412935324
epoch 2 batch id 401 loss 0.8198383450508118 train acc 0.7462593516209476
